In [12]:
# Load in our libraries
import pandas as pd
import numpy as np
from datetime import datetime
import math
# import re
# import random as random
# import xgboost as xgb

import seaborn as sns

# import warnings
# warnings.filterwarnings('ignore')

# # Going to use these 5 base models for the stacking
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.model_selection import KFold
# from sklearn.metrics import f1_score

# machine learning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# 評価関数
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

ModuleNotFoundError: No module named 'seaborn'

In [9]:
!pip install seaborn

In [ ]:
# 2. 築年月の数値を整数に変換し、NaNの処理を加えたdatetime形式変換関数を定義
def yyyymm_to_datetime(yyyymm_num):
    if math.isnan(yyyymm_num):  # NaNをチェック
        return None  # NaNの場合はNoneを返す
    yyyymm_int = int(yyyymm_num)  # 小数点以下を削除して整数に変換
    yyyymm_str = str(yyyymm_int)  # 整数を文字列に変換
    return datetime.strptime(yyyymm_str, "%Y%m")  # 文字列をdatetimeに変換

In [4]:
# 入力
PATH = 'C:/Users/y.kasashima/Desktop/signate/20241016/'
train = pd.read_csv(PATH + "train.csv")
test  = pd.read_csv(PATH + "test.csv")
full_data = [train, test]

C:\Users\y.kasashima\AppData\Local\Temp\ipykernel_10076\84277549.py:3: DtypeWarning: Columns (39,40,42,55,58) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(PATH + "train.csv")
C:\Users\y.kasashima\AppData\Local\Temp\ipykernel_10076\84277549.py:4: DtypeWarning: Columns (39,40,42,55,56,58,63) have mixed types. Specify dtype option on import or set low_memory=False.
  test  = pd.read_csv(PATH + "test.csv")


In [5]:
# 築経過月数の算出
current_date = datetime.now()
train['year_built_dt'] = train['year_built'].apply(yyyymm_to_datetime)
test['year_built_dt'] = test['year_built'].apply(yyyymm_to_datetime)
train['year_built_num'] = train['year_built_dt'].apply(lambda x: (current_date.year - x.year) * 12 + current_date.month - x.month)
test['year_built_num'] = test['year_built_dt'].apply(lambda x: (current_date.year - x.year) * 12 + current_date.month - x.month)

# 方角をワンホットエンコーディングで8変数追加
train = pd.get_dummies(train, columns=['snapshot_window_angle'], prefix='angle')
test = pd.get_dummies(test, columns=['snapshot_window_angle'], prefix='angle')

# 2. snapshot_window_angleをワンホットエンコーディングに変換
df_encoded = pd.get_dummies(train, columns=['building_structure'], prefix='bs')

In [6]:
# 追加済み変数
# ・unit_area : 専有面積
# ・year_built_num : 築年月数(築年月から経過月数を算出)
# ・room_count : 間取部屋数
# ・madori_number_all : 間取部屋数(代表)
# ・flg_new : 新築・未入居フラグ
# ・walk_distance1 : 徒歩距離1
# ・money_kyoueki : 共益費/管理費(代表)
# ・angle_4.0~angle_6.0 : 8方角を8種類の列として表現し、南東、南、南西の3種類
# ・convenience_distance : コンビニからの距離
# ・super_distance : スーパーからの距離
# ・hospital_distance : 総合病院からの距離
# ・room_kaisuu : 部屋階数
# ・building_structure : 建物構造
# ・land_chisei : 地勢
# ・money_rimawari_now : 現行利回り
cols = ['unit_area', 'year_built_num', 'room_count', 'madori_number_all', 'flg_new', 'walk_distance1', 'money_kyoueki', 'angle_4.0','angle_5.0', 'angle_6.0', 'convenience_distance', 'super_distance', 'hospital_distance', 'room_kaisuu', 'bs_1.0', 'bs_2.0', 'bs_3.0', 'land_chisei', 'money_rimawari_now', 'money_room']


In [ ]:
 
plt.figure(figsize=(2, 2)) # プロットサイズを設定
sns.pairplot(np.log(df_pickup), size=2.0) # df_pickupの変数の全ての組み合わせについて散布図を出力

In [17]:
train["dwelling_unit_window_angle"].head()

0    5.0
1    5.0
2    3.0
3    6.0
4    6.0
Name: dwelling_unit_window_angle, dtype: float64

In [7]:
import pandas as pd
from itertools import combinations

def create_interaction_features(df, target_column=None, columns=None):
    """
    特定のカラムを除外して相互作用カラムを作成し、最後に除外したカラムを戻す関数。
    target_columnが指定されない場合、そのカラムの処理はスキップされる。
    
    パラメータ:
    df: pandas DataFrame
        元のデータフレーム
    target_column: str, optional
        除外する目的変数の列名（例: 'money_room'）。デフォルトはNone。
    columns: list, optional
        掛け合わせたい列のリスト。デフォルトはNoneで、数値データ型の列が自動的に選択される。
    
    戻り値:
    pandas DataFrame: 元のデータフレームに新しい相互作用カラムを追加したもの
    """
    
    # 目的変数が指定されていれば一時的に除外
    if target_column is not None and target_column in df.columns:
        target_data = df[target_column]
        df = df.drop(columns=[target_column])
    else:
        target_data = None
    
    # 掛け合わせる列が指定されていない場合、数値データの列を自動選択
    if columns is None:
        columns = df.select_dtypes(include='number').columns
    
    # 2つずつの組み合わせを作成
    for col1, col2 in combinations(columns, 2):
        new_col_name = f"{col1}_x_{col2}"
        df[new_col_name] = df[col1] * df[col2]
    
    # 除外した目的変数がある場合は元に戻す
    if target_data is not None:
        df[target_column] = target_data
    
    return df

# 例としてのデータフレーム
data = {
    'money_room': [500, 700, 1000],
    'size': [30, 45, 60],
    'rooms': [2, 3, 4],
    'age': [10, 20, 30]
}

df = pd.DataFrame(data)

# 'money_room'を除外して相互作用カラムを作成
# 引数として target_column を指定する場合
df_with_interactions = create_interaction_features(df, target_column='money_room')
print("With target column excluded:")
print(df_with_interactions)

# # 引数として target_column を指定しない場合
# df_with_interactions_no_target = create_interaction_features(df)
# print("\nWithout target column exclusion:")
# print(df_with_interactions_no_target)


With target column excluded:
   size  rooms  age  size_x_rooms  size_x_age  rooms_x_age  money_room
0    30      2   10            60         300           20         500
1    45      3   20           135         900           60         700
2    60      4   30           240        1800          120        1000
